In [2]:
import pandas as pd
import numpy as np
import ast
#import seaborn
from collections import Counter
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer("english")
from bs4 import BeautifulSoup
import logging, gensim, bz2

ImportError: No module named gensim

In [4]:
def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

### Reading_Data

In [7]:
review_data = pd.read_csv("../data/yelp_academic_dataset_review.csv")

In [10]:
review_data.head(n=5)

,funny,user_id,review_id,text,business_id,stars,date,useful,type,cool
0,0,KpkOkG6RIf4Ra25Lhhxf1A,NxL8SIC5yqOdnlXCg18IBg,If you enjoy service by someone who is as comp...,2aFiy99vNLklCx3T_tGS9A,5,2011-10-10,0,review,0
1,0,bQ7fQq1otn9hKX-gXRsrgA,pXbbIgOXvLuTi_SPs1hQEQ,After being on the phone with Verizon Wireless...,2aFiy99vNLklCx3T_tGS9A,5,2010-12-29,1,review,0
2,0,r1NUhdNmL6yU9Bn-Yx6FTw,wslW2Lu4NYylb1jEapAGsw,Great service! Corey is very service oriented....,2aFiy99vNLklCx3T_tGS9A,5,2011-04-29,0,review,0
3,0,aW3ix1KNZAvoM8q-WghA3Q,GP6YEearUWrzPtQYSF1vVg,Highly recommended. Went in yesterday looking ...,2LfIuF3_sX6uwe-IR-P0jQ,5,2014-07-14,0,review,1
4,0,YOo-Cip8HqvKp_p9nEGphw,25RlYGq2s5qShi-pn3ufVA,I walked in here looking for a specific piece ...,2LfIuF3_sX6uwe-IR-P0jQ,4,2014-01-15,0,review,0


In [61]:
business_data = pd.read_csv("../data/yelp_academic_dataset_business.csv") 

In [62]:
business_data.shape

(144072, 16)

In [64]:
business_data.head(n=5)

,neighborhood,business_id,hours,is_open,address,attributes,categories,city,review_count,name,longitude,state,stars,latitude,postal_code,type
0,NaN,0DI8Dt2PJp07XkVvIElIcQ,"['Monday 11:0-21:0', 'Tuesday 11:0-21:0', 'Wed...",0,"227 E Baseline Rd, Ste J2","['BikeParking: True', 'BusinessAcceptsBitcoin:...","['Tobacco Shops', 'Nightlife', 'Vape Shops', '...",Tempe,17,Innovative Vapors,-111.936102,AZ,4.5,33.378214,85283,business
1,NaN,LTlCaCGZE14GuaUXUGbamg,"['Monday 0:0-0:0', 'Tuesday 0:0-0:0', 'Wednesd...",1,495 S Grand Central Pkwy,"['BusinessAcceptsBitcoin: False', 'BusinessAcc...","['Caterers', 'Grocery', 'Food', 'Event Plannin...",Las Vegas,9,Cut and Taste,-115.159272,NV,5.0,36.192284,89106,business
2,Dufferin Grove,EDqCEAGXVGCH4FJXgqtjqg,"['Monday 11:0-2:0', 'Tuesday 11:0-2:0', 'Wedne...",1,979 Bloor Street W,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Restaurants', 'Pizza', 'Chicken Wings', 'Ita...",Toronto,7,Pizza Pizza,-79.429089,ON,2.5,43.661054,M6H 1L5,business
3,NaN,cnGIivYRLxpF7tBVR_JwWA,"['Tuesday 10:0-21:0', 'Wednesday 10:0-21:0', '...",1,7014 Steubenville Pike,"['AcceptsInsurance: False', 'BusinessAcceptsCr...","['Hair Removal', 'Beauty & Spas', 'Blow Dry/Ou...",Oakdale,4,Plush Salon and Spa,-80.174540,PA,4.0,40.444544,15071,business
4,Downtown Core,cdk-qqJ71q6P7TJTww_DSA,NaN,1,321 Jarvis Street,"['BusinessAcceptsCreditCards: True', 'Restaura...","['Hotels & Travel', 'Event Planning & Services...",Toronto,8,Comfort Inn,-79.375401,ON,3.0,43.659829,M5B 2C2,business


### Getting Review Data for only Restaurants (Pittsburgh)

In [65]:
business_data = business_data.loc[business_data["categories"].notnull(),:]

In [66]:
def convert_list(x):
    return ast.literal_eval(x)
business_data["categories"] = map(convert_list,business_data["categories"])

In [69]:
category_labels = []
for categories in business_data["categories"]:
    category_labels += categories

In [70]:
category_dict = Counter(category_labels)
categorstaurants"]

In [73]:
Restaurant = [1 if "Restaurants" in categories else 0 for categories in business_data["categories"]]
business_data["Restaurant"] = Restaurant
np.sum(Restaurant)
float(np.sum(Restaurant))/business_data.shape[0]

In [94]:
restaurant_data = business_data.loc[business_data["Restaurant"]==1,:]
restaurant_review = pd.merge(review_data,restaurant_data,on="business_id")

In [108]:
#dump(restaurant_review,"../Pickled_files/restaurant_review.pkl")

### Load Data Start from here

In [10]:
restaurant_review = load("../Pickled_files/restaurant_review.pkl")

### Add your city and get the data

In [69]:
pitt_review = restaurant_review[restaurant_review["city"]=="Pittsburgh"]

[u'hello']

### Clean Reviews

In [68]:
import nltk
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
def preprocess_text(text):
    parse_text = BeautifulSoup(text).get_text()
    tokens = CountVectorizer(stop_words='english').build_tokenizer()(parse_text.lower())
    result = []
    
    
    for w in tokens:
        if w in stops:
            continue
        lem_word = wordnet_lemmatizer.lemmatize(w)
        stem_word = snowball_stemmer.stem(lem_word)
        result.append(stem_word)
    
    
    
    return result

In [ ]:
#restaurant_review["clean_text"] = map(preprocess_text,restaurant_review["text"])

### Map the function to the data of the city

In [1]:
pitt_review["clean_text"] = map(preprocess_text,pitt_review["text"])

In [74]:
pitt_review.shape

(100765, 27)

In [71]:
#dump(pitt_review,"../Picled_files/pitt_review.pkl")

In [3]:
pitt_review = load("../Pickled_files/pitt_review.pkl")

NameError: name 'load' is not defined

In [9]:
pitt_review[["stars_x","stars_y"]]

,stars_x,stars_y
1032,3,2.5
1033,3,2.5
1034,1,2.5
1035,1,2.5
1036,4,2.5
1037,3,2.5
1038,3,2.5
1039,3,2.5
1040,5,2.5
1041,1,2.5


### Preparing data for gensim for LDA

In [2]:
data = pitt_review["clean_text"]
dictionary = gensim.corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]

In [ ]:
#dump(corpus,"../Pickled_files/corpus.pkl")
#dump(dictionary,"../Pickled_files/dictionary.pkl")

### Checkin Data 

In [2]:
checkin_data = pd.read_csv("../data/yelp_academic_dataset_checkin.csv")

In [11]:
len(set(checkin_data["business_id"]))

125532

### Viz Word Trends

In [198]:
pitt_review = load("../Pickled_files/pitt_review.pkl")
pitt_review["word_freq"] = map(lambda x: Counter(x),pitt_review["clean_text"])

import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import matplotlib.dates as mdates
import matplotlib as mpl

def myFormatter(x, pos):
    return pd.to_datetime(x,format="%Y-%m").year

pitt_review["date"] = map(lambda x: pd.datetime.strptime(x, '%Y-%m-%d'),pitt_review["date"])
pitt_review["month"] = map(lambda x: x.month , pitt_review["date"])
pitt_review["year"] = map(lambda x: x.year , pitt_review["date"])
pitt_review["nreviews"] = 1 

words = ["call","deliv"]

def plot_word_trends(words):

    for word in words:
        pitt_review[word] = map(lambda x: x[word], pitt_review["word_freq"])

    feat = ["month","year","nreviews"] + words
    avgStatspitt = pitt_review[feat].groupby(by=["year","month"]).sum()


    for word in words:
        avgStatspitt[word + "prop"] = avgStatspitt[word]/avgStatspitt["nreviews"]
        avgStatspitt[word + "prop"] = avgStatspitt[word + "prop"].rolling(window = 5).mean()

    feat_prop = map(lambda x: x + "prop" ,words)
    avgStatspitt.reset_index(inplace=True)
    avgStatspitt = avgStatspitt[avgStatspitt["year"] > 2008]

    ym = zip(avgStatspitt["year"],avgStatspitt["month"])
    ym  = map(lambda x: str(x[0]) + "-" + str(x[1]),ym)
    avgStatspitt["date"] = map(lambda x: pd.datetime.strptime(x, '%Y-%m'),ym)

    pm = pd.melt(avgStatspitt, id_vars=['date'], value_vars=feat_prop)

    pm["Dummy"] = 0
    sns.set_style('ticks')
    fig, ax = plt.subplots()

    fig.set_size_inches(11.7, 8.27)
    sns.tsplot(data=pm, time="date", condition="variable",unit="Dummy", value="value",ax = ax)



    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(myFormatter))

    fig.autofmt_xdate()
    plt.show()

plot_word_trends(["food"])